In [ ]:
#| label: afiFig6jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path
import os
import sys
from contextlib import contextmanager
from repo2data.repo2data import Repo2Data

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(*os.path.normpath(os.getcwd()).split(os.path.sep)[:os.path.normpath(os.getcwd()).split(os.path.sep).index('mooc')+1])
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)

data_dir = dataset_path / "04-B1-02-AFI"
data_file = "fig6.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

params = mat_contents["params"]
n_range = mat_contents["n_range"][0]
r_analytical = mat_contents["r_analytical"]
r_approximation = mat_contents["r_approximation"]

## Plot
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}
# PYTHON CODE


init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"][0][0][0],
        y = abs(np.squeeze(np.asarray(r_analytical[ii]))),
        name = 'Analytical',
        text = 'Analytical',
        hoverinfo = 'x+y+text') for ii in range(len(n_range))]

data1[2]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"][0][0][0],
        y = abs(np.squeeze(np.asarray(r_approximation[ii]))),
        name = 'Approximation',
        text = 'Approximation',
        hoverinfo = 'x+y+text') for ii in range(len(n_range))]

data2[2]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(n_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(n_range[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 2,
    currentvalue = {"prefix": "n value: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Excitation Flip Angle (°)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='r = S<sub>2</sub>/S<sub>1</sub>',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['EXC_FA'][0][0][0][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)
